# Post-Trade Analytics (AQC-905)\n\nThis notebook is a lightweight companion to `tools/post_trade_analytics.py`.\n\nInputs:\n- A trade CSV (from `mei-backtester replay --export-trades`).\n- An events JSONL (from `artifacts/events/events.jsonl`).\n

In [ ]:
from pathlib import Path\nimport pandas as pd\nimport json\n\nTRADES_CSV = Path("/tmp/trades.csv")\nEVENTS_JSONL = Path("artifacts/events/events.jsonl")\n\ntrades = pd.read_csv(TRADES_CSV)\ntrades.head()\n

In [ ]:
# Symbol contribution\ntrades["pnl_net_usd"] = trades["pnl_usd"].fillna(0.0) - trades["fee_usd"].fillna(0.0)\nby_symbol = trades.groupby("symbol").agg(trades=("trade_id","count"), pnl_net_usd=("pnl_net_usd","sum")).sort_values("pnl_net_usd", ascending=False)\nby_symbol.head(20)\n

In [ ]:
# Entry audit events (ENTRY_*)\nevents = []\nif EVENTS_JSONL.exists():\n    for ln in EVENTS_JSONL.read_text(encoding="utf-8", errors="replace").splitlines():\n        s = ln.strip()\n        if not s:\n            continue\n        try:\n            obj = json.loads(s)\n        except Exception:\n            continue\n        if isinstance(obj, dict):\n            events.append(obj)\n\nentry = []\nfor e in events:\n    if str(e.get("schema")) != "aiq_event_v1":\n        continue\n    if str(e.get("kind", "")).lower() != "audit":\n        continue\n    data = e.get("data") if isinstance(e.get("data"), dict) else {}\n    ev = str(data.get("event", ""))\n    if not ev.startswith("ENTRY_"):\n        continue\n    entry.append({"ts": e.get("ts"), "symbol": e.get("symbol"), "event": ev})\n\nentry_df = pd.DataFrame(entry)\nentry_df.groupby("event").size().sort_values(ascending=False).head(30)\n